# **Deep Learning - AT&T**

## **Project**
One of the main pain point that AT&T users are facing is constant exposure to SPAM messages.

AT&T has been able to manually flag spam messages for a time, but they are looking for an automated way of detecting spams to protect their users.

## **Goals**
the goal is to build a spam detector, that can automatically flag spams as they come based solely on the sms' content. 
The study is structured into four key components:

1. **Initial explorations** of the data present in the dataset

2. **Data Cleaning and Preprocessing** 

3. **Models training** 

4. **Performances analysis**


In [384]:
# Import Spacy and English initialisation
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [385]:
# Library imports for project analysis

import pandas as pd
import io
import os
import re
import shutil
import string
import tensorflow as tf
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, GRU, LSTM
import spacy
import plotly.graph_objects as go

### 1. **Initial explorations** of the data present in the dataset

In [386]:
df = pd.read_csv("spam.csv", encoding='cp1252')

In [387]:
print("Number of rows and columns:  : {}".format(df.shape))
print()

print("Display of dataset: ")
display(df.head())
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Number of rows and columns:  : (5572, 5)

Display of dataset: 


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN



Basics statistics: 


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
count,5572,5572,50,12,6
unique,2,5169,43,10,5
top,ham,"Sorry, I'll call later","bt not his girlfrnd... G o o d n i g h t . . .@""","MK17 92H. 450Ppw 16""","GNT:-)"""
freq,4825,30,3,2,2



Percentage of missing values: 


v1             0.000000
v2             0.000000
Unnamed: 2    99.102656
Unnamed: 3    99.784637
Unnamed: 4    99.892319
dtype: float64

In [388]:
df["v1"].value_counts()

v1
ham     4825
spam     747
Name: count, dtype: int64

In [389]:
# Keep only the two first columns for analysis
df = df[["v1","v2"]]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### 2. **Data Cleaning and Preprocessing** 

In [390]:
# Import Stop words and en_core_web_sm (NLP model)
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [391]:
# Data Cleaning
df["v2_format_clean"] = df["v2"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
df["v2_format_clean"] = df["v2_format_clean"].apply(lambda x: x.replace(" +"," ").lower().strip())
df["v2_format_clean"] = df["v2_format_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) & (token.text not in STOP_WORDS)]))

display(100*df["v2_format_clean"].isnull().sum()/df.shape[0])

0.0

In [392]:
df.head()

,v1,v2,v2_format_clean
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think usf live


In [393]:
# Encode ham and spam into 0 and 1
label_encoder = LabelEncoder()
df['v1'] = label_encoder.fit_transform(df['v1'])
df.head()

,v1,v2,v2_format_clean
0,0,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,0,U dun say so early hor... U c already then say...,u dun early hor u c
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think usf live


In [394]:
# Encoding mails
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000) # instanciate the tokenizer
tokenizer.fit_on_texts(df["v2_format_clean"])
df["v2_encoded"] = tokenizer.texts_to_sequences(df.v2_format_clean)
df["len_v2"] = df["v2_encoded"].apply(lambda x: len(x))
df = df[df["len_v2"]!=0]

Tensorflow is incapable as of now to create a tensor dataset based on lists of different lengths, we will have to store all of our encoded texts into a single numpy array before creating the tensorflow dataset. Not all our sequences are the same length, this is where the tf.keras.preprocessing.sequence.pad_sequences comes in handy, it will add zero padding at the beginning (padding="pre") or at the end (padding="post") of your sequences so they all have equal length. Pad the sequences

In [395]:
df_pad = tf.keras.preprocessing.sequence.pad_sequences(df.v2_encoded, padding="post")

In [396]:
full_ds = tf.data.Dataset.from_tensor_slices((df_pad, df.v1.values))

In [397]:
# Train Test Split
TAKE_SIZE = int(0.7*df.shape[0])

train_data = full_ds.take(TAKE_SIZE).shuffle(TAKE_SIZE)
train_data = train_data.batch(64)

test_data = full_ds.skip(TAKE_SIZE)
test_data = test_data.batch(64)

for mail, spam_ham in train_data.take(1):
  print(mail, spam_ham)

tf.Tensor(
[[328 138  26 ...   0   0   0]
 [339  80   0 ...   0   0   0]
 [ 30  68 104 ...   0   0   0]
 ...
 [177   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]
 [830 352 831 ...   0   0   0]], shape=(64, 47), dtype=int32) tf.Tensor(
[0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1], shape=(64,), dtype=int64)


### 3. **Model training**

a) Simple RNN

In [398]:
vocab_size = 1000
model = tf.keras.Sequential([
                  # Word Embedding layer           
                  Embedding(vocab_size+1, 64, input_shape=[mail.shape[1],],name="embedding"),
                  # Gobal average pooling
                  SimpleRNN(units=64, return_sequences=True), # maintains the sequential nature
                  SimpleRNN(units=32, return_sequences=False), # returns the last output
                  # Dense layers once the data is flat
                  Dense(16, activation='relu'),
                  Dense(8, activation='relu'),

                  # output layer with as many neurons as the number of classes
                  # for the target variable and softmax activation
                  Dense(1, activation="sigmoid")
])

In [399]:
model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 64)            64064     
                                                                 
 simple_rnn_26 (SimpleRNN)   (None, 47, 64)            8256      
                                                                 
 simple_rnn_27 (SimpleRNN)   (None, 32)                3104      
                                                                 
 dense_96 (Dense)            (None, 16)                528       
                                                                 
 dense_97 (Dense)            (None, 8)                 136       
                                                                 
 dense_98 (Dense)            (None, 1)                 9         
                                                                 
Total params: 76097 (297.25 KB)
Trainable params: 760

In [400]:
optimizer= tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [401]:
# Weight Categories

weights = 1/(df["v1"]).value_counts()
weights = weights * len(df)/2
weights = {index : values for index , values in zip(weights.index,weights.values)}
weights

{0: 0.5798157274480394, 1: 3.632214765100671}

In [402]:
# Model training 
model.fit(train_data,
          epochs=20, 
          validation_data=test_data,
          class_weight=weights)

Epoch 1/20


60/60 [==============================] - 1s 12ms/step - loss: 0.4363 - binary_accuracy: 0.8503 - val_loss: 0.2419 - val_binary_accuracy: 0.9520
Epoch 2/20
60/60 [==============================] - 1s 10ms/step - loss: 0.2056 - binary_accuracy: 0.9535 - val_loss: 0.2150 - val_binary_accuracy: 0.9501
Epoch 3/20
60/60 [==============================] - 1s 10ms/step - loss: 0.1345 - binary_accuracy: 0.9704 - val_loss: 0.1482 - val_binary_accuracy: 0.9624
Epoch 4/20
60/60 [==============================] - 1s 10ms/step - loss: 0.1232 - binary_accuracy: 0.9747 - val_loss: 0.1737 - val_binary_accuracy: 0.9495
Epoch 5/20
60/60 [==============================] - 1s 10ms/step - loss: 0.1132 - binary_accuracy: 0.9823 - val_loss: 0.0910 - val_binary_accuracy: 0.9778
Epoch 6/20
60/60 [==============================] - 1s 10ms/step - loss: 0.1062 - binary_accuracy: 0.9583 - val_loss: 0.2815 - val_binary_accuracy: 0.9292
Epoch 7/20
60/60 [==============================] - 1s 10ms/step - loss: 0.0693 -

In [403]:
model.save("./model/model_simpleRNN.h5")
import json
json.dump(model.history.history, open("./history/simpleRNN_history.json", 'w'))

/Users/simoncld/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



b) GRU

In [404]:
vocab_size = 1000
model_gru = tf.keras.Sequential([
                  Embedding(vocab_size+1, 64, input_shape=[mail.shape[1],],name="embedding"),
                  GRU(units=64, return_sequences=True), # maintains the sequential nature
                  GRU(units=32, return_sequences=False), # returns the last output
                  Dense(16, activation='relu'),
                  Dense(8, activation='relu'),

                  Dense(1, activation="sigmoid")
])

In [405]:
model_gru.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 64)            64064     
                                                                 
 gru_18 (GRU)                (None, 47, 64)            24960     
                                                                 
 gru_19 (GRU)                (None, 32)                9408      
                                                                 
 dense_99 (Dense)            (None, 16)                528       
                                                                 
 dense_100 (Dense)           (None, 8)                 136       
                                                                 
 dense_101 (Dense)           (None, 1)                 9         
                                                                 
Total params: 99105 (387.13 KB)
Trainable params: 991

In [406]:
optimizer= tf.keras.optimizers.Adam()

model_gru.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [407]:
model_gru.fit(train_data,
              epochs=20, 
              validation_data=test_data,
              class_weight=weights)

Epoch 1/20


60/60 [==============================] - 3s 27ms/step - loss: 0.6927 - binary_accuracy: 0.7878 - val_loss: 0.6822 - val_binary_accuracy: 0.8615
Epoch 2/20
60/60 [==============================] - 1s 23ms/step - loss: 0.6924 - binary_accuracy: 0.6750 - val_loss: 0.6920 - val_binary_accuracy: 0.8615
Epoch 3/20
60/60 [==============================] - 1s 24ms/step - loss: 0.6925 - binary_accuracy: 0.5253 - val_loss: 0.6896 - val_binary_accuracy: 0.8615
Epoch 4/20
60/60 [==============================] - 1s 24ms/step - loss: 0.6925 - binary_accuracy: 0.4351 - val_loss: 0.6948 - val_binary_accuracy: 0.1392
Epoch 5/20
60/60 [==============================] - 1s 23ms/step - loss: 0.6924 - binary_accuracy: 0.2693 - val_loss: 0.6932 - val_binary_accuracy: 0.1392
Epoch 6/20
60/60 [==============================] - 1s 23ms/step - loss: 0.6924 - binary_accuracy: 0.5106 - val_loss: 0.6938 - val_binary_accuracy: 0.1392
Epoch 7/20
60/60 [==============================] - 1s 23ms/step - loss: 0.6924 -

In [408]:
model_gru.save("./model/model_gru.h5")
import json
json.dump(model_gru.history.history, open("./history/GRU_history.json", 'w'))

c) LSTM

In [409]:
vocab_size = 1000

model_lstm = tf.keras.Sequential([
                  Embedding(vocab_size+1, 64, input_shape=[mail.shape[1],],name="embedding"),
                  LSTM(units=64, return_sequences=True), # maintains the sequential nature
                  LSTM(units=32, return_sequences=False), # returns the last output
                  Dense(16, activation='relu'),
                  Dense(8, activation='relu'),

                  Dense(1, activation="sigmoid", name="last")
])


In [410]:
model_lstm.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 64)            64064     
                                                                 
 lstm_30 (LSTM)              (None, 47, 64)            33024     
                                                                 
 lstm_31 (LSTM)              (None, 32)                12416     
                                                                 
 dense_102 (Dense)           (None, 16)                528       
                                                                 
 dense_103 (Dense)           (None, 8)                 136       
                                                                 
 last (Dense)                (None, 1)                 9         
                                                                 
Total params: 110177 (430.38 KB)
Trainable params: 11

In [411]:
optimizer= tf.keras.optimizers.Adam()

model_lstm.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [412]:
model_lstm.fit(train_data,
              epochs=100, 
              validation_data=test_data,
              class_weight=weights)

Epoch 1/100


60/60 [==============================] - 4s 34ms/step - loss: 0.6930 - binary_accuracy: 0.2133 - val_loss: 0.6875 - val_binary_accuracy: 0.8615
Epoch 2/100
60/60 [==============================] - 2s 29ms/step - loss: 0.6927 - binary_accuracy: 0.7833 - val_loss: 0.6936 - val_binary_accuracy: 0.1392
Epoch 3/100
60/60 [==============================] - 2s 28ms/step - loss: 0.6928 - binary_accuracy: 0.3765 - val_loss: 0.6907 - val_binary_accuracy: 0.8615
Epoch 4/100
60/60 [==============================] - 2s 28ms/step - loss: 0.6898 - binary_accuracy: 0.8157 - val_loss: 0.6659 - val_binary_accuracy: 0.3602
Epoch 5/100
60/60 [==============================] - 2s 28ms/step - loss: 0.2661 - binary_accuracy: 0.9570 - val_loss: 0.1807 - val_binary_accuracy: 0.9791
Epoch 6/100
60/60 [==============================] - 2s 28ms/step - loss: 0.1858 - binary_accuracy: 0.9741 - val_loss: 0.1428 - val_binary_accuracy: 0.9784
Epoch 7/100
60/60 [==============================] - 2s 29ms/step - loss: 0.

In [413]:
model_lstm.save("./model/model_lstm.h5")
import json
json.dump(model_lstm.history.history, open("./history/LSTM_history.json", 'w'))

d) LSTM with Attention

In [414]:
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Attention
from tensorflow.keras.models import Model

# Définition de l'entrée
inputs = Input(shape=(mail.shape[1],))

# Embedding
embedding_layer = Embedding(vocab_size+1, 64, input_length=mail.shape[1], name="embedding")
embedding_output = embedding_layer(inputs)

# First LSTM
lstm_output = LSTM(units=64, return_sequences=True)(embedding_output)

# Attention
attention_output = Attention()([lstm_output, lstm_output])

# Second LSTM with Attention
lstm_with_attention_output = LSTM(units=32, return_sequences=False)(attention_output)

dense_output = Dense(16, activation='relu')(lstm_with_attention_output)
dense_output = Dense(8, activation='relu')(dense_output)

outputs = Dense(1, activation="sigmoid", name="last")(dense_output)

# Model
model_LSTM_with_attention = Model(inputs=inputs, outputs=outputs)

model_LSTM_with_attention.summary()


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 47)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 47, 64)               64064     ['input_8[0][0]']             
                                                                                                  
 lstm_32 (LSTM)              (None, 47, 64)               33024     ['embedding[0][0]']           
                                                                                                  
 attention_7 (Attention)     (None, 47, 64)               0         ['lstm_32[0][0]',             
                                                                     'lstm_32[0][0]']       

In [415]:
optimizer= tf.keras.optimizers.Adam()

model_LSTM_with_attention.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [416]:
model_LSTM_with_attention.fit(train_data,
              epochs=100, 
              validation_data=test_data,
              class_weight=weights)

Epoch 1/100


2024-02-08 18:41:38.932581: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


59/60 [============================>.] - ETA: 0s - loss: 0.6088 - binary_accuracy: 0.8612

2024-02-08 18:41:41.093645: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 10 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


60/60 [==============================] - 3s 35ms/step - loss: 0.6076 - binary_accuracy: 0.8617 - val_loss: 0.2551 - val_binary_accuracy: 0.9754
Epoch 2/100
60/60 [==============================] - 2s 33ms/step - loss: 0.2046 - binary_accuracy: 0.9736 - val_loss: 0.1985 - val_binary_accuracy: 0.9655
Epoch 3/100
60/60 [==============================] - 2s 32ms/step - loss: 0.2010 - binary_accuracy: 0.9644 - val_loss: 0.1503 - val_binary_accuracy: 0.9735
Epoch 4/100
60/60 [==============================] - 2s 30ms/step - loss: 0.2437 - binary_accuracy: 0.9723 - val_loss: 0.2738 - val_binary_accuracy: 0.9704
Epoch 5/100
60/60 [==============================] - 2s 31ms/step - loss: 0.2137 - binary_accuracy: 0.9739 - val_loss: 0.1756 - val_binary_accuracy: 0.9711
Epoch 6/100
60/60 [==============================] - 2s 31ms/step - loss: 0.4509 - binary_accuracy: 0.7033 - val_loss: 0.6036 - val_binary_accuracy: 0.4957
Epoch 7/100
60/60 [==============================] - 2s 32ms/step - loss: 0.

In [417]:
model_LSTM_with_attention.save("./model/model_LSTM_with_attention.h5")
import json
json.dump(model_LSTM_with_attention.history.history, open("./history/LSTM_att_history.json", 'w'))

### 4. **Performance analysis**

a) RNN

In [418]:
simpleRNN_history = json.load(open("./history/simpleRNN_history.json", 'r'))
model_simpleRNN = tf.keras.models.load_model("./model/model_simpleRNN.h5")
fig = go.Figure()
fig.add_trace(go.Scatter(y=simpleRNN_history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=simpleRNN_history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()


b) GRU

In [419]:
GRU_history = json.load(open("./history/GRU_history.json", 'r'))
model_gru = tf.keras.models.load_model("./model/model_gru.h5")

fig = go.Figure()
fig.add_trace(go.Scatter(y=GRU_history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=GRU_history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()

c) LSTM

In [420]:
LSTM_history = json.load(open("./history/LSTM_history.json", 'r'))
model_lstm = tf.keras.models.load_model("./model/model_lstm.h5")

fig = go.Figure()
fig.add_trace(go.Scatter(y=LSTM_history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=LSTM_history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()


d) LSTM with Attention

In [422]:
LSTM_wa_history = json.load(open("./history/LSTM_att_history.json", 'r'))
model_lstm = tf.keras.models.load_model("./model/model_LSTM_with_attention.h5")

fig = go.Figure()
fig.add_trace(go.Scatter(y=LSTM_wa_history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=LSTM_wa_history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()


### Conclusion 


Based on the provided models for predicting whether emails are spam or ham, let's analyze their performances and conclude on the best choice considering overfitting and other factors.

* RNN Model:

    Achieves a high binary accuracy of around 99.39% on the training set and 95.63% on the validation set.
    Appears to generalize well with a consistent improvement in validation accuracy across epochs.
    Shows minimal signs of overfitting with a slight increase in validation loss towards the end but still maintains a high accuracy.

* GRU Model:

    Starts with relatively poor performance, with binary accuracy dropping significantly during training.
    Doesn't appear to learn effectively, as indicated by the low accuracy on both training and validation sets.
    Demonstrates signs of severe overfitting with a large gap between training and validation accuracies.

* LSTM Model:

    Initially achieves a high binary accuracy of around 86.17% on the training set and 97.54% on the validation set.
    Maintains high accuracy throughout training but shows signs of slight overfitting with a slight divergence between training and validation accuracies.
    However, it successfully learns the patterns in the data and generalizes well.

* LSTM with Attention Model:

    Starts with a reasonable binary accuracy of around 86.12% on the training set and 97.48% on the validation set.
    Maintains high accuracy throughout training with minimal signs of overfitting, similar to the basic LSTM model.
    Incorporates attention mechanism, which might help in focusing on important parts of the input sequence, potentially improving performance.


Considering the performances and potential for overfitting, the LSTM with Attention model seems to be the most promising choice. It achieves high accuracy on both training and validation sets while demonstrating resilience against overfitting. The attention mechanism adds an additional layer of interpretability and could potentially enhance performance further.